In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
train = pd.read_csv('X_train_final.csv')
test = pd.read_csv('X_test_final.csv')

comunicaciones_conectividad_list(multilabel: One hot encode)


In [3]:
# Remove '_1' suffix from all column names
test.columns = test.columns.str.replace(r'_1$', '', regex=True)

# Define a function to process both train and test
def process_multilabel_column(df, column_name):
    # Split strings into lists, drop NAs
    df[f'{column_name}_list'] = df[column_name].str.split(',')

    # Clean whitespace
    df[f'{column_name}_list'] = df[f'{column_name}_list'].apply(
        lambda x: [i.strip() for i in x if i.strip()] if isinstance(x, list) else []
    )

    return df

# Step 1: Process both datasets
train = process_multilabel_column(train, 'comunicaciones_conectividad')
test = process_multilabel_column(test, 'comunicaciones_conectividad')

# Step 2: Fit MultiLabelBinarizer on combined features
mlb = MultiLabelBinarizer()

# Combine the lists first so that both train and test have same set of columns
combined_lists = pd.concat([
    train['comunicaciones_conectividad_list'],
    test['comunicaciones_conectividad_list']
], ignore_index=True)

mlb.fit(combined_lists)

# Step 3: Transform each dataset
train_ohe = pd.DataFrame(
    mlb.transform(train['comunicaciones_conectividad_list']),
    columns=mlb.classes_,
    index=train.index
)
test_ohe = pd.DataFrame(
    mlb.transform(test['comunicaciones_conectividad_list']),
    columns=mlb.classes_,
    index=test.index
)

# Step 4: Concatenate back
train = pd.concat([train, train_ohe], axis=1)
test = pd.concat([test, test_ohe], axis=1)

train = train.drop(['Unnamed: 0','título','gráfica_memoria_gráfica','comunicaciones_conectividad_list','comunicaciones_conectividad'], axis=1)
test = test.drop(['Unnamed: 0', 'título','gráfica_memoria_gráfica','comunicaciones_conectividad_list','comunicaciones_conectividad'], axis=1)

train['otras_características_fecha_lanzamiento'] = train['otras_características_fecha_lanzamiento'].astype(float).astype('Int64')

KeyError: "['Unnamed: 0'] not found in axis"

In [4]:
def group_operating_systems(train_df, test_df, os_column='sistema_operativo_sistema_operativo'):
    """
    Group operating system values into major categories.
    
    Parameters:
    -----------
    train_df : pandas DataFrame
        Training dataset
    test_df : pandas DataFrame
        Test dataset
    os_column : str
        Name of the operating system column
    
    Returns:
    --------
    train_df_processed : pandas DataFrame
        Training dataset with grouped OS column
    test_df_processed : pandas DataFrame
        Test dataset with grouped OS column
    """
    # Make copies to avoid modifying original dataframes
    train_df_processed = train_df.copy()
    test_df_processed = test_df.copy()
    
    # Check if the OS column exists in datasets
    if os_column not in train_df_processed.columns:
        print(f"Warning: '{os_column}' column not found in training dataset")
        return train_df_processed, test_df_processed
    
    # Define OS grouping mapping
    os_mapping = {
        # Windows 11 group
        'Windows 11': [
            'Microsoft Windows 11 Home', 
            'Windows 11 Professional', 
            'Windows 11 Home', 
            'Microsoft Windows 11 Professional',
            'Windows 11 S',
            'Microsoft Windows 11 IoT Enterprise'
        ],
        
        # Windows 10 group
        'Windows 10': [
            'Windows 10 Home', 
            'Windows 10 Professional', 
            'Microsoft Windows 10 Professional',
            'Windows 10 S',
            'Microsoft Windows 10 Home',
            'Microsoft Windows 10'
        ],
        
        # Older Windows group
        'Windows Other': [
            'Microsoft Windows 7 Professional',
            'Microsoft Windows 8 Professional',
            'Windows 8'
        ],
        
        # macOS group
        'macOS': [
            'macOS Sequoia',
            'macOS Big Sur',
            'macOS Ventura',
            'macOS Sonoma',
            'Mac OS X',
            'macOS Monterey',
            'macOS Catalina',
            'macOS Sierra'
        ],
        
        # Chrome OS group
        'Chrome OS': [
            'Chrome OS',
            'Google Chrome OS'
        ],
        
        # DOS-like group
        'DOS': [
            'FreeDOS',
            'Free DOS',
            'DOS'
        ],
        
        # Linux-based group
        'Linux': [
            'Linux',
            'Endless OS'
        ],
        
        # Thin client OS group
        'Thin Client OS': [
            'Wyse Thin OS',
            'HP ThinPro',
            'IGEL OS'
        ],
        
        # Mobile OS group
        'Mobile OS': [
            'Google Android'
        ],
        
        # No OS group
        'No OS': [
            'ninguno',
            'sin sistema operativo'
        ],
        
        # Other
        'Other': [
            'configurable'
        ]
    }
    
    # Create a reverse mapping for easier lookup
    reverse_mapping = {}
    for category, os_list in os_mapping.items():
        for os in os_list:
            reverse_mapping[os] = category
    
    # Function to map OS values to categories
    def map_os(os_value):
        if pd.isna(os_value):
            return np.nan
        
        # Direct lookup in reverse mapping
        if os_value in reverse_mapping:
            return reverse_mapping[os_value]
        
        # Partial matching for Windows, macOS, etc.
        if 'Windows 11' in os_value:
            return 'Windows 11'
        elif 'Windows 10' in os_value:
            return 'Windows 10'
        elif 'Windows' in os_value:
            return 'Windows Other'
        elif 'macOS' in os_value or 'Mac OS' in os_value:
            return 'macOS'
        elif 'Chrome' in os_value:
            return 'Chrome OS'
        elif 'DOS' in os_value or 'FreeDOS' in os_value:
            return 'DOS'
        elif 'Linux' in os_value:
            return 'Linux'
        elif 'Android' in os_value:
            return 'Mobile OS'
        elif os_value in ['ninguno', 'sin sistema operativo']:
            return 'No OS'
        
        # Default to Other for unrecognized values
        return 'Other'
    
    # Create a new column with the grouped OS values
    new_column = 'os_group'
    train_df_processed[new_column] = train_df_processed[os_column].apply(map_os)
    
    if os_column in test_df_processed.columns:
        test_df_processed[new_column] = test_df_processed[os_column].apply(map_os)
    else:
        print(f"Warning: '{os_column}' column not found in test dataset")
    
    # Print summary of the grouping
    if os_column in train_df_processed.columns:
        original_unique = train_df_processed[os_column].nunique()
        grouped_unique = train_df_processed[new_column].nunique()
        print(f"OS Grouping: {original_unique} unique values → {grouped_unique} groups")
        
        # Show group distribution
        group_counts = train_df_processed[new_column].value_counts()
        print("\nOS group distribution in training data:")
        for group, count in group_counts.items():
            print(f"- {group}: {count} instances ({count/len(train_df_processed)*100:.1f}%)")
    
    return train_df_processed, test_df_processed

# Apply the OS grouping to both datasets
train, test = group_operating_systems(train, test)



TypeError: argument of type 'float' is not iterable

In [6]:
def group_operating_systems_consolidated(train_df, test_df, os_column='sistema_operativo_sistema_operativo'):
    """
    Create a more consolidated grouping of operating systems with better balance.
    
    Parameters:
    -----------
    train_df : pandas DataFrame
        Training dataset
    test_df : pandas DataFrame
        Test dataset
    os_column : str
        Name of the operating system column
    
    Returns:
    --------
    train_df_processed : pandas DataFrame
        Training dataset with consolidated OS groups
    test_df_processed : pandas DataFrame
        Test dataset with consolidated OS groups
    """
    # Make copies to avoid modifying original dataframes
    train_df_processed = train_df.copy()
    test_df_processed = test_df.copy()
    
    # Check if the OS column exists in datasets
    if os_column not in train_df_processed.columns:
        print(f"Warning: '{os_column}' column not found in training dataset")
        return train_df_processed, test_df_processed
    
    # Function to consolidate OS values into fewer, more balanced groups
    def consolidate_os(os_value):
        if pd.isna(os_value):
            return np.nan
            
        os_str = str(os_value).lower()
        
        # Group all Windows versions together
        if 'windows' in os_str:
            return 'Windows'
            
        # Group all macOS versions together
        if 'macos' in os_str or 'mac os' in os_str:
            return 'macOS'
            
        # Group all systems with no OS
        if os_str in ['ninguno', 'sin sistema operativo', 'no os']:
            return 'No OS'
            
        # Group all DOS variants
        if 'dos' in os_str or 'freedos' in os_str:
            return 'DOS'
            
        # Group all other systems into "Other OS"
        if os_str in ['', 'nan', 'none', 'null']:
            return np.nan
            
        return 'Other OS'
    
    # Create a new column with the consolidated OS groups
    new_column = 'os_consolidated'
    
    # Apply consolidation to training data
    if os_column in train_df_processed.columns:
        train_df_processed[new_column] = train_df_processed[os_column].apply(consolidate_os)
    
    # Apply consolidation to test data
    if os_column in test_df_processed.columns:
        test_df_processed[new_column] = test_df_processed[os_column].apply(consolidate_os)
    
    # One-hot encode the consolidated OS categories
    train_dummies = pd.get_dummies(train_df_processed[new_column], prefix='os')
    
    # Handle test set one-hot encoding (if the column exists)
    if os_column in test_df_processed.columns:
        test_dummies = pd.get_dummies(test_df_processed[new_column], prefix='os')
        
        # Ensure test has all the same columns as train
        for col in train_dummies.columns:
            if col not in test_dummies.columns:
                test_dummies[col] = 0
        
        # Ensure columns are in the same order
        test_dummies = test_dummies[train_dummies.columns]
        
        # Add the one-hot encoded columns to test dataframe
        test_df_processed = pd.concat([test_df_processed, test_dummies], axis=1)
    
    # Add the one-hot encoded columns to train dataframe
    train_df_processed = pd.concat([train_df_processed, train_dummies], axis=1)
    
    # Print summary of the grouping
    if os_column in train_df_processed.columns:
        original_unique = train_df_processed[os_column].nunique()
        consolidated_unique = train_df_processed[new_column].nunique()
        print(f"OS Consolidation: {original_unique} unique values → {consolidated_unique} groups")
        
        # Show group distribution
        group_counts = train_df_processed[new_column].value_counts()
        total_rows = len(train_df_processed)
        
        print("\nConsolidated OS group distribution in training data:")
        for group, count in group_counts.items():
            print(f"- {group}: {count} instances ({count/total_rows*100:.1f}%)")
        
        print("\nCreated one-hot encoded columns:")
        for col in train_dummies.columns:
            count = train_dummies[col].sum()
            print(f"- {col}: {count} instances ({count/total_rows*100:.1f}%)")
    
    return train_df_processed, test_df_processed

# Apply the consolidated OS grouping to both datasets
train, test = group_operating_systems_consolidated(train, test)
train = train.drop(['os_consolidated','os_group','sistema_operativo_sistema_operativo'], axis=1)
test = test.drop(['os_consolidated','os_group','sistema_operativo_sistema_operativo'], axis=1)


OS Consolidation: 37 unique values → 5 groups

Consolidated OS group distribution in training data:
- Windows: 3448 instances (54.4%)
- No OS: 541 instances (8.5%)
- DOS: 357 instances (5.6%)
- macOS: 175 instances (2.8%)
- Other OS: 97 instances (1.5%)

Created one-hot encoded columns:
- os_DOS: 357 instances (5.6%)
- os_No OS: 541 instances (8.5%)
- os_Other OS: 97 instances (1.5%)
- os_Windows: 3448 instances (54.4%)
- os_macOS: 175 instances (2.8%)


Combine medidas_alto_cm and altura_mm

In [7]:
# First, convert cm to mm (1 cm = 10 mm)
train['medidas_alto_cm_in_mm'] = train['medidas_alto_cm'] * 10
test['medidas_alto_cm_in_mm'] = test['medidas_alto_cm'] * 10


# Now combine both columns into one — prefer mm if it's present
train['altura_mm'] = train['medidas_altura_mm'].fillna(train['medidas_alto_cm_in_mm'])
test['altura_mm'] = test['medidas_altura_mm'].fillna(test['medidas_alto_cm_in_mm'])


# Drop the original columns if you don't need them
train.drop(['medidas_altura_mm', 'medidas_alto_cm', 'medidas_alto_cm_in_mm'], axis=1, inplace=True)
test.drop(['medidas_altura_mm', 'medidas_alto_cm', 'medidas_alto_cm_in_mm'], axis=1, inplace=True)

In [8]:
# Standardize 'Mini DisplayPort' and 'Mini-DisplayPort' into one label
train['gráfica_salida_vídeo'] = train['gráfica_salida_vídeo'].replace({
    'Mini-DisplayPort': 'Mini DisplayPort'
})
# Standardize 'Mini DisplayPort' and 'Mini-DisplayPort' into one label
test['gráfica_salida_vídeo'] = test['gráfica_salida_vídeo'].replace({
    'Mini-DisplayPort': 'Mini DisplayPort'
})

In [9]:
col_info = []

for col in train.columns:
    dtype = train[col].dtype
    if dtype == 'object':
        uniques = train[col].unique()
        col_info.append((col, 'object', uniques))
    else:
        col_info.append((col, 'numeric'))

# Example: printing the result
for item in col_info:
    if item[1] == 'object':
        print(f"{item[0]} (object): {item[2]}")
    else:
        print(f"{item[0]} (numeric)")


disco_duro_tipo_disco_duro (object): ['disco duro M.2 SSD' 'disco duro SSD' nan 'SSD' 'PCIe SSD'
 'sin disco duro' 'disco duro HDD' 'memoria flash' 'SATA'
 'disco híbrido (HHD)']
procesador_frecuencia_turbo_máx__GHz (numeric)
tipo_producto (object): ['Mini PC' 'PC gaming' 'Portátil profesional' 'Workstation'
 'Portátil gaming' 'Portátil multimedia' 'PC de oficina' 'PC multimedia'
 'Barebone' nan 'Chromebook' 'Portátil convertible' 'Ultrabook' 'Netbook'
 'Thin Client' 'PC completo' 'Kit ampliación PC' 'Portátil 3D']
tipo (object): ['Desktop' 'Laptop']
medidas_profundidad_cm (numeric)
sonido_número_altavoces (numeric)
medidas_peso_kg (numeric)
gráfica_salida_vídeo (object): [nan 'Thunderbolt 4' 'HDMI 2.0' 'HDMI' 'HDMI 2.1' 'USB-C' 'VGA'
 'Thunderbolt 3' 'DisplayPort' 'Mini DisplayPort' 'HDMI 1.4' 'DVI'
 'Mini HDMI' 'Micro HDMI']
disco_duro_capacidad_memoria_ssd_GB (numeric)
procesador_número_hilos_ejecución (numeric)
procesador_tdp_W (numeric)
procesador_número_núcleos_procesador_cores (

In [10]:
X_train = train
X_test = test
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

Fillout missing value with 'Missing_value' for label encoding

In [11]:
# List of columns you want to fill
cols_to_fill = [
    'gráfica_tarjeta_gráfica',
    'pantalla_tecnología_pantalla',
    'procesador_name'
]

# Fill missing values with 'Missing_value'
for col in cols_to_fill:
    X_train[col] = X_train[col].fillna('Missing_value')
    X_test[col] = X_test[col].fillna('Missing_value')

Actually label encode. 

In [12]:
from sklearn.preprocessing import LabelEncoder

# Columns to label encode
label_cols = [
    'company_name',
    'gráfica_tarjeta_gráfica',
    'pantalla_tecnología_pantalla',
    'procesador_name'
]

# Apply LabelEncoder for each column
for col in label_cols:
    le = LabelEncoder()
    # Fit on train and test combined to handle unseen labels safely
    combined = pd.concat([X_train[col], X_test[col]], axis=0).astype(str)
    le.fit(combined)

    # Transform
    X_train[col + '_label'] = le.transform(X_train[col].astype(str))
    X_test[col + '_label'] = le.transform(X_test[col].astype(str))

X_train = X_train.drop(label_cols, axis=1)
X_test = X_test.drop(label_cols, axis=1)


One-hot encoding

In [13]:
# List of columns to one-hot encode
one_hot_cols = [
    'procesador_nivel_caché',
    'tipo_producto',
    'disco_duro_tipo_disco_duro',
    'gráfica_salida_vídeo',
    'ram_tipo_ram',
    'almacenamiento_lector_óptico',
    'tipo',
    'color'
]

# Function to one-hot encode with prefix and include NaNs as a category
def one_hot_encode_with_na(df, columns):
    return pd.get_dummies(df, columns=columns, prefix=columns, dummy_na=True)

# Apply to both train and test
X_train_encoded = one_hot_encode_with_na(X_train, one_hot_cols)
X_test_encoded = one_hot_encode_with_na(X_test, one_hot_cols)

# Align train and test columns (make sure both have same one-hot columns)
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)


Rounding your price column to 2 decimal places

In [14]:
y_train['price_avg'] = y_train['price_avg'].round(2)
y_test['price_avg'] = y_test['price_avg'].round(2)  # If your test set also has it


In [15]:
y_train['price_avg'].describe()


count     6341.000000
mean      1512.945026
std       1236.036577
min        100.060000
25%        746.200000
50%       1180.970000
75%       1881.790000
max      17749.900000
Name: price_avg, dtype: float64

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(y_train['price_avg'], bins=50, kde=True)
plt.title("Price Distribution")
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.show()


ModuleNotFoundError: No module named 'seaborn'

Since the price is skewed, we will transform it into log.

In [17]:
y_train = np.log1p(y_train['price_avg'])
y_test = np.log1p(y_test['price_avg'])

In [18]:
plt.figure(figsize=(10, 5))
sns.histplot(y_train, bins=50, kde=True)
plt.title("Log-Transformed Price Distribution")
plt.xlabel("Log(1 + Price)")
plt.ylabel("Frequency")
plt.show()

NameError: name 'sns' is not defined

<Figure size 1000x500 with 0 Axes>

In [19]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error ,mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import KFold


xgb_model = xgb.XGBRegressor(
    missing=np.nan,
    objective='reg:squarederror',  # Important for regression
    random_state=42
)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # Use RMSE
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all cores
)

grid_search.fit(X_train_encoded, y_train)


print("Best parameters:", grid_search.best_params_)
print("Best RMSE score (negative):", grid_search.best_score_)

# Refit the best model
best_model = grid_search.best_estimator_

# Predict on test
y_pred_log = best_model.predict(X_test_encoded)
rmse = mean_squared_error(y_test, y_pred_log)
print(f"Validation RMSE: {rmse:.4f}")
print("MAE:", mean_absolute_error(y_test, y_pred_log))
print("RMSLE:", mean_squared_log_error(y_test, y_pred_log) ** 0.5)



ModuleNotFoundError: No module named 'xgboost'

In [ ]:

y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test)

real_mae = mean_absolute_error(y_test_real, y_pred_real)
real_mse = mean_squared_error(y_test_real, y_pred_real)
real_rmse = np.sqrt(real_mse)

print(f"Real RMSE: {real_rmse:.2f}")
print(f"Real MAE: {real_mae:.2f}")


Real RMSE: 660.48
Real MAE: 302.10


In [ ]:
average_price = y_test_real.mean()
mae_percent = (299.02 / average_price) * 100
rmse_percent = (638.10 / average_price) * 100
print(f"MAE as % of average price: {mae_percent:.2f}%")
print(f"RMSE as % of average price: {rmse_percent:.2f}%")


MAE as % of average price: 19.68%
RMSE as % of average price: 42.00%


# KNN


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer
k = 5
# Make a copy so you don't mess up your original dataset
X_knn = X_train_encoded.copy()

# Fill all remaining NaNs with a placeholder like -1 or median/most_frequent
imputer = SimpleImputer(strategy='most_frequent')  
X_knn_imputed = imputer.fit_transform(X_knn)


knn = NearestNeighbors(n_neighbors=k, metric='euclidean')  # or 'cosine'
knn.fit(X_knn_imputed)  # Use same encoded dataset you used in XGBoost


NearestNeighbors(metric='euclidean')

In [ ]:
X_knn.to_csv('temp.csv')

In [ ]:
query = X_knn_imputed[0].reshape(1, -1)  # for example, first row
distances, indices = knn.kneighbors(query)
predicted_prices = best_model.predict(X_knn_imputed[indices[0]])
real_prices = y_train.iloc[indices[0]].values

# Recommendation system and price prediction of it.

In [ ]:
def recommend_and_predict(user_input, knn_model, ml_model, X_train, y_train, top_k=5):
    # 1. Find top-K similar products
    distances, indices = knn_model.kneighbors(user_input.reshape(1, -1), n_neighbors=top_k)
    similar_products = X_train.iloc[indices[0]]
    
    # 2. Predict price
    predicted_prices = ml_model.predict(user_input.reshape(1, -1))
    

    similar_predicted_prices = ml_model.predict(similar_products)
    real_prices = y_train.iloc[indices[0]].values

    return similar_products, predicted_prices, similar_predicted_prices, real_prices


In [ ]:
ex = [20.0, 3, 15.0, 3.5, 1.0, 16.0, 4.0, 2800.0, 10.0, 244.0, 24.0, 35.0, 36.0, 2022, 38.0,16.0, 400.0, 16.0, 4.0, 7.5, 0,0,0,0,0,0,1,0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1.6, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 16.0, 13, 52, 14, 48, 0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0]
ex_array = np.array(ex)
print(recommend_and_predict(ex_array,knn,best_model,X_knn,y_train,5))

(      medidas_profundidad_cm  ofertas_count  \
4771                    17.0              1   
3127                    17.0              1   
4394                    18.0              7   
2457                    18.0              1   
4334                    18.0              1   

      procesador_número_hilos_ejecución  procesador_frecuencia_turbo_máx__GHz  \
4771                               12.0                                   3.8   
3127                                6.0                                   3.7   
4394                               12.0                                   3.6   
2457                               12.0                                   3.6   
4334                                4.0                                   3.7   

      disco_duro_número_discos_duros_instalados  medidas_ancho_cm  \
4771                                        1.0              17.0   
3127                                        1.0              17.0   
4394                  

In [ ]:
X_train_encoded.to_csv("temp.csv")